In [1]:
from omg.datamodule import DataModule
from omg.datamodule.dataloader import OMGTorchDataset
from torch_geometric.loader import DataLoader

In [2]:
import torch

In [3]:
from omg.model.model_utils import SinusoidalTimeEmbeddings

In [4]:
dataset = DataModule.from_lmdb(["./../data/mp_20/train.lmdb"])

example_data = OMGTorchDataset(dataset, convert_to_fractional=True)

loader = DataLoader(example_data, batch_size=10)

2024-09-26 23:36:59.874 | INFO     | omg.datamodule.datamodule:add_from_lmdb:832 - Reusing existing LMDB file: dataset_274518dcfedeeb30399261097eb4fcb7.lmdb


In [5]:
for batch in loader:
    t = torch.rand(1)
    print(t,batch)
    break   

tensor([0.8366]) OMGDataBatch(
  n_atoms=[10],
  batch=[90],
  species=[90],
  cell=[10, 3, 3],
  pos=[90, 3],
  property={},
  ptr=[11]
)


In [6]:
prop = torch.rand(10)
prop.shape
prop.double()

tensor([0.1999, 0.7823, 0.9393, 0.2577, 0.5047, 0.6132, 0.2943, 0.6279, 0.9250,
        0.6363], dtype=torch.float64)

In [7]:
prop_embd = SinusoidalTimeEmbeddings(32)
time_embd = SinusoidalTimeEmbeddings(64)

In [8]:
prop = prop_embd(prop)

In [9]:
prop.shape

torch.Size([10, 32])

In [10]:
t = t.repeat(10)
t.dtype

torch.float32

In [11]:
t = time_embd(t)

In [12]:
from omg.model.encoders.cspnet_full import CSPNetFull

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
model = CSPNetFull(latent_dim=64)
model = model.double()

In [14]:
prop = prop.double()


In [15]:
prop.dtype

torch.float64

In [16]:
t.shape

torch.Size([10, 64])

In [17]:
md = model(batch,t = t, prop = prop)

property is here
property is here
property is here
property is here
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
here


In [18]:
md


Data(species_b=[90, 100], species_eta=[90, 100], pos_b=[90, 3], pos_eta=[90, 3], cell_b=[10, 3, 3], cell_eta=[10, 3, 3])

In [19]:
batch.species

tensor([11, 11, 11, 25, 27, 28,  8,  8,  8,  8,  8,  8, 60, 13, 13, 13, 13, 13,
        13, 13, 13, 29, 29, 29, 29, 48, 49, 49, 49, 52, 52, 52, 52, 33, 44, 45,
        45, 45, 45, 23, 23, 23, 23,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 70, 50, 56, 56, 48, 55, 37, 37, 69, 17, 17, 17, 17, 17, 17, 67, 14,
        28, 28, 28, 28, 19, 19, 37, 69, 23, 23,  8,  8,  8,  8,  8,  8,  8,  8])

In [20]:
md.species_b

tensor([[ 0.2302,  0.5970,  0.2352,  ...,  0.6971, -0.0596, -0.2588],
        [ 0.2325,  0.6011,  0.2312,  ...,  0.6939, -0.0606, -0.2638],
        [ 0.2291,  0.6027,  0.2353,  ...,  0.6958, -0.0616, -0.2628],
        ...,
        [ 0.6844, -0.1052,  0.0366,  ..., -1.3878, -0.2221,  0.3213],
        [ 0.6833, -0.1059,  0.0345,  ..., -1.3900, -0.2227,  0.3252],
        [ 0.6869, -0.1021,  0.0320,  ..., -1.3876, -0.2262,  0.3237]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [21]:
md


Data(species_b=[90, 100], species_eta=[90, 100], pos_b=[90, 3], pos_eta=[90, 3], cell_b=[10, 3, 3], cell_eta=[10, 3, 3])